In [18]:
import pandas as pd
import pymysql
import numpy as np
from tabulate import tabulate

In [15]:
df = pd.read_csv(r"F:\vscode\traffic_stops.csv")
print(df.head(5))


    stop_date stop_time country_name driver_gender  driver_age_raw  \
0  2020-01-01   0:00:00       Canada             M              59   
1  2020-01-01   0:01:00        India             M              35   
2  2020-01-01   0:02:00          USA             M              26   
3  2020-01-01   0:03:00       Canada             M              26   
4  2020-01-01   0:04:00       Canada             M              62   

   driver_age driver_race     violation_raw violation  search_conducted  \
0          19       Asian     Drunk Driving  Speeding              True   
1          58       Other             Other     Other             False   
2          76       Black  Signal Violation  Speeding             False   
3          76       Black          Speeding       DUI              True   
4          75       Other          Speeding     Other             False   

      search_type stop_outcome  is_arrested stop_duration  drugs_related_stop  \
0  Vehicle Search       Ticket         True    

C:\Users\MANEE\AppData\Local\Temp\ipykernel_14780\4135857335.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"F:\vscode\traffic_stops.csv")


In [16]:
print("Before filling nulls:\n", df.isnull().sum())

Before filling nulls:
 stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age_raw            0
driver_age                0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64


In [19]:
# Replace NaN with None for MySQL compatibility
df = df.replace({np.nan: None})

In [20]:
#Convert date and time columns properly
df['stop_date'] = pd.to_datetime(df['stop_date'], errors='coerce').dt.date
df['stop_time'] = pd.to_datetime(df['stop_time'], errors='coerce').dt.time

# Fill missing times as None (MySQL NULL)
df['stop_time'] = df['stop_time'].apply(lambda x: x if pd.notnull(x) else None)

# Fill missing search_type values
df['search_type'] = df['search_type'].apply(lambda x: x if x is not None else 'unknown')

print("🧹 Data cleaned successfully!")

C:\Users\MANEE\AppData\Local\Temp\ipykernel_14780\1427496722.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['stop_time'] = pd.to_datetime(df['stop_time'], errors='coerce').dt.time


🧹 Data cleaned successfully!


In [21]:
# 4️⃣ Connect to MySQL using pymysql
try:
    conn_mysql = pymysql.connect(
        host="localhost",
        user="root",
        password="572001",
        autocommit=True
    )
    cursor_mysql = conn_mysql.cursor()
    print("✅ MySQL connection established using pymysql!")
except Exception as e:
    print("❌ Error connecting to MySQL:", e)
    exit()

✅ MySQL connection established using pymysql!


In [23]:
cursor_mysql.execute("CREATE DATABASE IF NOT EXISTS Digital_Ledger_db;")
cursor_mysql.execute("USE Digital_Ledger_db;")

cursor_mysql.execute("""
CREATE TABLE IF NOT EXISTS Digital_Ledger_data (
   stop_date DATE,
   stop_time TIME,
   country_name VARCHAR(50),
   driver_gender VARCHAR(50),
   driver_age_raw INT,
   driver_age INT,
   driver_race VARCHAR(50),
   violation_raw VARCHAR(50),
   violation VARCHAR(50),
   search_conducted BOOLEAN,
   search_type VARCHAR(50),
   stop_outcome VARCHAR(50),
   is_arrested BOOLEAN,
   stop_duration VARCHAR(50),
   drugs_related_stop BOOLEAN,
   vehicle_number VARCHAR(50)
);
""")
conn_mysql.commit()
print("✅ Table 'Digital_Ledger_data' created successfully!")

✅ Table 'Digital_Ledger_data' created successfully!


In [24]:
data_list = df.values.tolist()
insert_query = """
INSERT INTO Digital_Ledger_data (
    stop_date, stop_time, country_name, driver_gender,
    driver_age_raw, driver_age, driver_race, violation_raw,
    violation, search_conducted, search_type, stop_outcome,
    is_arrested, stop_duration, drugs_related_stop, vehicle_number
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

try:
    cursor_mysql.executemany(insert_query, data_list)
    conn_mysql.commit()
    print("✅ Data inserted successfully!")
except Exception as e:
    print("❌ Error during data insertion:", e)

✅ Data inserted successfully!


In [25]:
from tabulate import tabulate

In [26]:
# 1. Top 10 vehicle numbers involved in drug-related stops
query1 = """
SELECT vehicle_number 
FROM Digital_Ledger_data 
WHERE drugs_related_stop = TRUE 
LIMIT 10;
"""
cursor_mysql.execute(query1)
print("\n🔹 Top 10 vehicles (drug-related):")
print(tabulate(cursor_mysql.fetchall()))


🔹 Top 10 vehicles (drug-related):
----------
UP76DY3473
RJ83PZ4441
RJ32OM7264
RJ76TI3807
DL75KZ7835
DL50PO5101
UP67CQ9426
KA61JB1004
WB70IV9884
WB75TF1052
----------


In [27]:
# 2. Vehicles most frequently searched
query2 = """
SELECT vehicle_number, COUNT(*) AS search_count
FROM Digital_Ledger_data
GROUP BY vehicle_number
ORDER BY search_count DESC
LIMIT 1;
"""
cursor_mysql.execute(query2)
print("\n🔹 Most frequently searched vehicle:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Most frequently searched vehicle:
----------  -
UP76DY3473  1
----------  -


In [28]:
# 3. Driver age group with highest arrest rate
query3 = """
SELECT driver_age, COUNT(*) AS total
FROM Digital_Ledger_data
WHERE is_arrested = TRUE
GROUP BY driver_age
ORDER BY total DESC
LIMIT 1;
"""
cursor_mysql.execute(query3)
print("\n🔹 Highest arrest rate by driver age:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Highest arrest rate by driver age:
--  ---
28  580
--  ---


In [29]:
# 4. Gender distribution of drivers stopped in each country
query4 = """
SELECT country_name, driver_gender, COUNT(*) AS total_stops
FROM Digital_Ledger_data
GROUP BY country_name, driver_gender
ORDER BY country_name, total_stops DESC;
"""
cursor_mysql.execute(query4)
print("\n🔹 Gender distribution by country:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Gender distribution by country:
------  -  -----
Canada  F  10997
Canada  M  10911
India   F  11043
India   M  10955
USA     F  10841
USA     M  10791
------  -  -----


In [30]:
# 5. Race & gender with most searches
query5 = """
SELECT driver_race, driver_gender, COUNT(*) AS Count1
FROM Digital_Ledger_data
WHERE search_conducted = TRUE
GROUP BY driver_race, driver_gender
ORDER BY Count1 DESC LIMIT 1;
"""
cursor_mysql.execute(query5)
print("\n🔹 Most searched race & gender:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Most searched race & gender:
-----  -  ----
Asian  F  3410
-----  -  ----


In [31]:
# 7. Time of day with most stops
query7 = """
SELECT HOUR(stop_time) AS hour_of_day, COUNT(*) AS stop_count
FROM Digital_Ledger_data
GROUP BY HOUR(stop_time)
ORDER BY stop_count DESC
LIMIT 1;
"""
cursor_mysql.execute(query7)
print("\n🔹 Time of day with most stops:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Time of day with most stops:
-  ----
0  2760
-  ----


In [32]:
# 8. Average stop duration for violations
query8 = """
SELECT violation, AVG(CAST(stop_duration AS SIGNED)) AS avg_stop_duration, COUNT(*) AS total_stops
FROM Digital_Ledger_data
GROUP BY violation
ORDER BY avg_stop_duration DESC;
"""
cursor_mysql.execute(query8)
print("\n🔹 Avg stop duration by violation:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Avg stop duration by violation:
--------  -------  -----
Other     15.744   13194
Seatbelt  15.3344  13007
Speeding  15.2867  13150
DUI       15.2349  13075
Signal    15.202   13112
--------  -------  -----


In [33]:
# 9. Arrest rate by time of day
query9 = """
SELECT 
    CASE 
        WHEN HOUR(stop_time) BETWEEN 20 AND 23 THEN 'Night'
        WHEN HOUR(stop_time) BETWEEN 0 AND 4 THEN 'Late Night'
        ELSE 'Daytime'
    END AS time_period,
    COUNT(*) AS total_stops,
    SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) AS arrests,
    ROUND(SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) / COUNT(*) * 100, 2) AS arrest_rate_percent
FROM Digital_Ledger_data
GROUP BY time_period
ORDER BY arrest_rate_percent DESC;
"""
cursor_mysql.execute(query9)
print("\n🔹 Arrest rate by time period:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Arrest rate by time period:
----------  -----  -----  -----
Daytime     40938  20647  50.43
Night       10800   5419  50.18
Late Night  13800   6780  49.13
----------  -----  -----  -----


In [34]:
# 10. Violations most associated with searches/arrests
query10 = """
SELECT violation, COUNT(*) AS hg
FROM Digital_Ledger_data
WHERE search_conducted = TRUE OR is_arrested = TRUE
GROUP BY violation
ORDER BY hg DESC LIMIT 3;
"""
cursor_mysql.execute(query10)
print("\n🔹 Top 3 violations (search/arrest):")
print(tabulate(cursor_mysql.fetchall()))


🔹 Top 3 violations (search/arrest):
--------  ----
Speeding  9868
DUI       9867
Other     9857
--------  ----


In [35]:
# 11. Most common violations among drivers under 25
query11 = """
SELECT violation, COUNT(*) AS aba
FROM Digital_Ledger_data
WHERE driver_age < 25
GROUP BY violation
ORDER BY aba DESC LIMIT 1;
"""
cursor_mysql.execute(query11)
print("\n🔹 Most common violation (<25):")
print(tabulate(cursor_mysql.fetchall()))


🔹 Most common violation (<25):
--------  ----
Speeding  1476
--------  ----


In [36]:
# 12. Violations rarely resulting in search/arrest
query12 = """
SELECT violation, COUNT(*) AS aba
FROM Digital_Ledger_data
WHERE search_conducted = TRUE OR is_arrested = TRUE
GROUP BY violation
HAVING aba < 10
ORDER BY aba LIMIT 1;
"""
cursor_mysql.execute(query12)
records = cursor_mysql.fetchall()
if not records:
    print("\n🔹 Rare violations: NONE")
else:
    print("\n🔹 Rare violations:")
    print(tabulate(records))


🔹 Rare violations: NONE


In [37]:
# 13. Countries with highest drug-related stops
query13 = """
SELECT country_name, COUNT(*) AS ds
FROM Digital_Ledger_data
WHERE drugs_related_stop = TRUE
GROUP BY country_name
ORDER BY ds DESC LIMIT 5;
"""
cursor_mysql.execute(query13)
print("\n🔹 Top 5 countries (drug-related):")
print(tabulate(cursor_mysql.fetchall()))


🔹 Top 5 countries (drug-related):
------  -----
Canada  10976
India   10897
USA     10896
------  -----


In [38]:
# 14. Arrest rate by country and violation
query14 = """
SELECT country_name, violation,
    SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) AS arrest_count,
    COUNT(*) AS total_count,
    ROUND(SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) / COUNT(*) * 100, 2) AS arrest_rate_percent
FROM Digital_Ledger_data
GROUP BY country_name, violation
ORDER BY arrest_rate_percent DESC LIMIT 5;
"""
cursor_mysql.execute(query14)
print("\n🔹 Arrest rate by country & violation:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Arrest rate by country & violation:
------  --------  ----  ----  -----
Canada  DUI       2257  4414  51.13
India   DUI       2247  4420  50.84
India   Speeding  2212  4358  50.76
India   Signal    2251  4444  50.65
USA     Seatbelt  2145  4241  50.58
------  --------  ----  ----  -----


In [39]:
# 15. Country with most searches conducted
query15 = """
SELECT country_name, COUNT(*) AS df
FROM Digital_Ledger_data
WHERE search_conducted = TRUE
GROUP BY country_name
ORDER BY df DESC LIMIT 1;
"""
cursor_mysql.execute(query15)
print("\n🔹 Country with most searches:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Country with most searches:
------  -----
Canada  11020
------  -----


In [40]:
# C1. Yearly Breakdown of Stops and Arrests by Country
queryc1 = """
SELECT 
    country_name,
    year,
    COUNT(*) AS total_stops,
    SUM(is_arrested) AS total_arrests,
    SUM(SUM(is_arrested)) OVER (PARTITION BY country_name ORDER BY year) AS cumulative_arrests
FROM (
    SELECT country_name, YEAR(stop_date) AS year, is_arrested
    FROM Digital_Ledger_data
) AS sub
GROUP BY country_name, year
ORDER BY country_name, year;
"""
cursor_mysql.execute(queryc1)
print("\n🔹 Yearly breakdown of stops & arrests:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Yearly breakdown of stops & arrests:
------  ----  -----  -----  -----
Canada  2020  21908  10999  10999
India   2020  21998  11091  11091
USA     2020  21632  10756  10756
------  ----  -----  -----  -----


In [41]:
# C2. Violation pattern by driver age group and race
queryc2 = """
SELECT 
    v.driver_age_group,
    v.driver_race,
    v.violation,
    COUNT(*) AS total_violations
FROM (
    SELECT 
        driver_age,
        driver_race,
        violation,
        CASE 
            WHEN driver_age < 25 THEN 'Under 25'
            WHEN driver_age BETWEEN 25 AND 40 THEN '25-40'
            WHEN driver_age BETWEEN 41 AND 60 THEN '41-60'
            ELSE '60+'
        END AS driver_age_group
    FROM Digital_Ledger_data
) AS v
GROUP BY v.driver_age_group, v.driver_race, v.violation
ORDER BY v.driver_age_group, v.driver_race, total_violations DESC;
"""
cursor_mysql.execute(queryc2)
print("\n🔹 Violation pattern by age group & race:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Violation pattern by age group & race:
--------  --------  --------  ---
25-40     Asian     Signal    704
25-40     Asian     Speeding  697
25-40     Asian     DUI       682
25-40     Asian     Other     678
25-40     Asian     Seatbelt  672
25-40     Black     Seatbelt  670
25-40     Black     DUI       669
25-40     Black     Speeding  665
25-40     Black     Signal    659
25-40     Black     Other     639
25-40     Hispanic  Seatbelt  677
25-40     Hispanic  Other     672
25-40     Hispanic  DUI       667
25-40     Hispanic  Speeding  661
25-40     Hispanic  Signal    655
25-40     Other     Speeding  681
25-40     Other     DUI       675
25-40     Other     Seatbelt  668
25-40     Other     Other     667
25-40     Other     Signal    615
25-40     White     Signal    682
25-40     White     Other     672
25-40     White     Seatbelt  644
25-40     White     Speeding  643
25-40     White     DUI       633
41-60     Asian     Other     832
41-60     Asian     Signal    828
41-60 

In [42]:
# C3. Stop pattern by year, month, and hour
queryc3 = """
SELECT 
    t.year,
    t.month,
    t.hour,
    COUNT(*) AS total_stops
FROM (
    SELECT
        YEAR(stop_date) AS year,
        MONTH(stop_date) AS month,
        HOUR(stop_time) AS hour
    FROM Digital_Ledger_data
) AS t
GROUP BY t.year, t.month, t.hour
ORDER BY t.year, t.month, t.hour;
"""
cursor_mysql.execute(queryc3)
print("\n🔹 Stop pattern by year, month, hour:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Stop pattern by year, month, hour:
----  -  --  ----
2020  1   0  1860
2020  1   1  1860
2020  1   2  1860
2020  1   3  1860
2020  1   4  1860
2020  1   5  1860
2020  1   6  1860
2020  1   7  1860
2020  1   8  1860
2020  1   9  1860
2020  1  10  1860
2020  1  11  1860
2020  1  12  1860
2020  1  13  1860
2020  1  14  1860
2020  1  15  1860
2020  1  16  1860
2020  1  17  1860
2020  1  18  1860
2020  1  19  1860
2020  1  20  1860
2020  1  21  1860
2020  1  22  1860
2020  1  23  1860
2020  2   0   900
2020  2   1   900
2020  2   2   900
2020  2   3   900
2020  2   4   900
2020  2   5   900
2020  2   6   900
2020  2   7   900
2020  2   8   900
2020  2   9   900
2020  2  10   900
2020  2  11   900
2020  2  12   858
2020  2  13   840
2020  2  14   840
2020  2  15   840
2020  2  16   840
2020  2  17   840
2020  2  18   840
2020  2  19   840
2020  2  20   840
2020  2  21   840
2020  2  22   840
2020  2  23   840
----  -  --  ----


In [43]:
# C4. Top 10 violations by arrest rate
queryc4 = """
SELECT 
    violation,
    COUNT(*) AS total_stops,
    SUM(search_conducted) AS total_searches,
    SUM(is_arrested) AS total_arrests,
    ROUND(100 * SUM(search_conducted) / COUNT(*), 2) AS search_rate,
    ROUND(100 * SUM(is_arrested) / COUNT(*), 2) AS arrest_rate,
    RANK() OVER (ORDER BY ROUND(100 * SUM(is_arrested) / COUNT(*), 2) DESC) AS arrest_rank
FROM Digital_Ledger_data
GROUP BY violation
ORDER BY arrest_rate DESC
LIMIT 10;
"""
cursor_mysql.execute(queryc4)
print("\n🔹 Top 10 violations by arrest rate:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Top 10 violations by arrest rate:
--------  -----  ----  ----  -----  -----  -
DUI       13075  6481  6610  49.57  50.55  1
Seatbelt  13007  6562  6573  50.45  50.53  2
Speeding  13150  6573  6598  49.98  50.17  3
Signal    13112  6509  6549  49.64  49.95  4
Other     13194  6576  6516  49.84  49.39  5
--------  -----  ----  ----  -----  -----  -


In [44]:
# C5. Country-level driver demographics
queryc5 = """
SELECT 
    country_name,
    ROUND(AVG(driver_age), 1) AS avg_age,
    COUNT(DISTINCT driver_gender) AS gender_diversity,
    COUNT(DISTINCT driver_race) AS race_diversity,
    COUNT(*) AS total_drivers
FROM Digital_Ledger_data
GROUP BY country_name
ORDER BY total_drivers DESC;
"""
cursor_mysql.execute(queryc5)
print("\n🔹 Country-level demographics:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Country-level demographics:
------  ----  -  -  -----
India   49.1  2  5  21998
Canada  49.1  2  5  21908
USA     49.1  2  5  21632
------  ----  -  -  -----


In [47]:
queryc6 = """
SELECT 
    violation,
    COUNT(*) AS total_stops,
    SUM(is_arrested) AS total_arrests,
    ROUND(100 * SUM(is_arrested) / COUNT(*), 2) AS arrest_rate
FROM Digital_Ledger_data
GROUP BY violation
ORDER BY arrest_rate DESC
LIMIT 5;
"""
cursor_mysql.execute(queryc6)
print("\n🔹 Violations with highest arrest rate:")
print(tabulate(cursor_mysql.fetchall()))


🔹 Violations with highest arrest rate:
--------  -----  ----  -----
DUI       13075  6610  50.55
Seatbelt  13007  6573  50.53
Speeding  13150  6598  50.17
Signal    13112  6549  49.95
Other     13194  6516  49.39
--------  -----  ----  -----


In [48]:
cursor_mysql.close()
conn_mysql.close()
print("\n✅ MySQL connection closed successfully!")


✅ MySQL connection closed successfully!
